# Orange CNN (based on Pasquadibisceglie et al., 2020) 

Adapted from the original code by: 

- Pasquadibisceglie, V., Appice, A., Castellano, G., Malerba, D., & Modugno, G. (2020). Orange: Outcome-oriented predictive process monitoring based on image encoding and CNNs. IEEE Access, 8, 184073–184086. https://doi.org/10.1109/ACCESS.2020.3029323

github link: https://github.com/vinspdb/ORANGE

This notebook contains the code that tests an Orange CNN model originally created by Pasquadibisceglie et al. on the data provided by the Catharina hospital

## General imports

In [3]:
#general imports
import sys
import numpy
import pandas as pd
import numpy as np
from itertools import product
import datetime
from datetime import datetime
import time
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
from keras.preprocessing.sequence import pad_sequences
from glob import glob
pd.options.mode.chained_assignment = None  
import warnings
warnings.filterwarnings("ignore")

#hyperas imports for hyperparameter optimization
from hyperas import optim
from hyperas.distributions import choice, uniform
from hyperopt import Trials, STATUS_OK, tpe

seed = 42
np.random.seed(seed)
from tensorflow.compat.v1 import set_random_seed
set_random_seed(seed)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, MaxPooling2D, BatchNormalization, Conv2D, Activation, GlobalMaxPooling2D
from tensorflow.keras import regularizers, losses
from tensorflow.keras import backend as K
from tensorflow.keras.metrics import Precision, Recall, AUC, Accuracy, MeanSquaredError, MeanAbsoluteError
from tensorflow_addons.metrics import F1Score
import tensorflow as tf
tf.config.run_functions_eagerly(True)

from sklearn import metrics 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.preprocessing import StandardScaler
from sklearn import feature_selection
from sklearn.compose import ColumnTransformer
from imblearn.over_sampling import RandomOverSampler
from other_lib import globalvar
from other_lib.auk_score import AUK
from other_lib.general_functions import prepare_dataset_for_model, find_all_csv_locations, image_encoder_cv, image_encoder_val
from orange_lib import DeepInsight_train_norm


## Building the model
code use to build the prediction model

In [4]:
def create_model(x_train, x_val, x_test, y_train, y_val, y_test, binary):

    #convert input data to image data
    x_train_images, x_val_images, x_test_images = image_encoder_val(x_train, x_val, x_test, y_train, y_val, y_test)
    
    #build sequential model
    model = Sequential()
    input_shape = (x_train_images.shape[1], x_train_images.shape[1], 1)
    model.add(Conv2D({{choice([32, 64])}}, (2, 2), input_shape=input_shape, padding='same', kernel_initializer='glorot_uniform', kernel_regularizer=regularizers.l2({{choice([0.001, 0.0001])}})))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D({{choice([32, 64])}}, (4, 4), padding='same', kernel_initializer='glorot_uniform', kernel_regularizer=regularizers.l2({{choice([0.001, 0.0001])}})))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(GlobalMaxPooling2D())
    
    optimizer = tf.keras.optimizers.Adam(learning_rate={{choice([10 ** -4, 10 ** -3, 10 ** -2])}}, clipnorm=1.)   

    #add output layer based on the prediction type        
    if binary:
        model.add(Dense(1, activation='sigmoid'))
        model.compile(optimizer=optimizer, loss='binary_crossentropy',
                      metrics=['accuracy', globalvar.f1, globalvar.precision, globalvar.recall, globalvar.auc])
    else:
        model.add(Dense(1, activation="linear"))
        model.compile(optimizer=optimizer, loss='mae', metrics=['mae', 'mse', 'mape'])
        
    model.summary()
    
    earlystop = EarlyStopping(monitor='val_loss', min_delta=0.000001, patience=10, verbose=0, mode='min')
    lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=7, verbose=0, mode='auto',
                                       min_delta=0.0001, cooldown=0, min_lr=0)
    callbacks_list = [earlystop, lr_reducer]

    model.fit(x_train_images, y_train, epochs={{choice([50,100])}}, 
              batch_size={{choice([50, 100])}}, verbose=0, 
              callbacks=callbacks_list, 
              validation_data=(x_val_images, y_val))
    
    score = model.evaluate(x_test_images, y_test, verbose=0)
    print('score evaluated: ', score)
    print('binary: ', binary)
    
    if binary:
        f1 = score[2]
        return {'loss': -f1, 'status': STATUS_OK, 'model': model} #take the negative of f1 here since objective is to minimize and f1 usually maens higher is better
    else:
        mae = score[1]
        return {'loss': mae, 'status': STATUS_OK, 'model': model} #dont take negative value here since you want to minimize the mae

## With best model, calculate cv scores
function below is used to crossvalidate the results

In [5]:
def cross_validate_best_model(X, y, best_model, best_run, binary, output_dir, model_name, model_type):
    if binary: 
        kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42) #cannot do stratifiedkfold for regression tasks
        cv_accuracy_scores = []
        cv_f1_scores = []
        cv_precision_scores = []
        cv_recall_scores = []
        cv_auc_scores = []
        cv_auk_scores = []
    else:
        kfold = KFold(n_splits=5, shuffle=True, random_state=42) #regular kfold here
        cv_mae_scores = []
        cv_mse_scores = []
        cv_mape_scores = []

    callbacks_list = [globalvar.earlystop]
    fold_counter = 1

    for train, test in kfold.split(X, y): #cross validation to obtain stable results, only have to do padded_X since padded_X1 has same 
        print('Now starting fold: {} for model: {}'.format(fold_counter, model_name))
        
        x_train = X.loc[train]
        x_test = X.loc[test]
        y_train = y.loc[train]
        y_test = y.loc[test]

        #fill NaN value with mean of training data for both train and test data. Cant do mean per group since many groups have no data at all
        x_train.fillna(x_train.mean(), inplace=True)
        x_test.fillna(x_train.mean(), inplace=True)

        #scaling for non-additional features, only on train/test data to prevent data leakage, complete X returned without scaling
        additional_features = ['MedicationCode_B01AA04', 'MedicationCode_B01AA07', 'MedicationCode_B01AE07', 'MedicationCode_B01AF01', 
                               'MedicationCode_B01AF02', 'MedicationCode_B01AF03', 'MedicationCode_N02AJ13', 'MedicationCode_N02BE01',
                               'PlannedDuration', 'Duration', 'MedicationType', 'NOAC', 'MedicationStatus', 'temperature', 
                               'bloodPressure', 'Test_Hemoglobine', 'Test_eGFR', 'Test_INR', 'Test_Trombocyten']

        scaler = StandardScaler()    

        if 'tokenized' in model_name and 'transformer' not in model_type: #means all columns need to be encoded, regardless of additional or not
            x_train = pd.DataFrame(scaler.fit_transform(x_train))
            x_test = pd.DataFrame(scaler.fit_transform(x_test))
        elif 'additional' in model_name.lower() and 'ae_agg' not in model_name.lower(): #means only the additionally added columns need to be scaled
            x_train[additional_features] = scaler.fit_transform(x_train[additional_features])
            x_test[additional_features] = scaler.fit_transform(x_test[additional_features])

        #For lstm models, the input needs to be 3d instead of 2d. Therefore, add another dimension to the data
        if model_type == 'lstm' or model_type=='transformer':
            x_train = np.expand_dims(x_train, -1)
            x_test= np.expand_dims(x_test, -1)
        
            
        #convert input data to image data
        x_train_images, x_test_images = image_encoder_cv(x_train, x_test, y_train, y_test)
            
        best_model.fit(x_train_images,
                       y_train, 
                       epochs=best_run['epochs'], 
                       callbacks=callbacks_list, 
                       batch_size=best_run['batch_size'],
                       verbose=0)

        scores = best_model.evaluate(x_test_images, y_test, verbose=0)

        if binary:
            y_pred = best_model.predict(x_test_images, verbose=0)
            scores.append(AUK(y_test, y_pred.flatten()).calculate_auk()) #add AUK scores
            
            print("%s: %.2f%%" % (best_model.metrics_names[1], scores[1] * 100)) #accuracy of the test prediction
            cv_accuracy_scores.append(scores[1])
            cv_f1_scores.append(scores[2])
            cv_precision_scores.append(scores[3])
            cv_recall_scores.append(scores[4])
            cv_auc_scores.append(scores[5])
            cv_auk_scores.append(scores[6])
        else:
            print('{} score: {}'.format(best_model.metrics_names[1], scores[1]))
            cv_mae_scores.append(scores[1])
            cv_mse_scores.append(scores[2])
            cv_mape_scores.append(scores[3])

        fold_counter += 1 #update fold counter

    #calculate measures
    if binary:
        print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cv_accuracy_scores)*100, numpy.std(cv_accuracy_scores)*100))
        measures = [numpy.mean(cv_accuracy_scores), 
                    numpy.std(cv_accuracy_scores),
                    numpy.mean(cv_f1_scores), 
                    numpy.std(cv_f1_scores),
                    numpy.mean(cv_precision_scores),
                    numpy.std(cv_precision_scores),
                    numpy.mean(cv_recall_scores), 
                    numpy.std(cv_recall_scores),
                    numpy.mean(cv_auc_scores), 
                    numpy.std(cv_auc_scores),
                    numpy.mean(cv_auk_scores),
                    numpy.std(cv_auk_scores)] #average over all splits
    else:
        print('average mae score over all splits: {} (+/- {}%)'.format(numpy.mean(cv_mae_scores), numpy.std(cv_mae_scores)))
        measures = [numpy.mean(cv_mae_scores),
                    numpy.std(cv_mae_scores),
                    numpy.mean(cv_mse_scores),
                    numpy.std(cv_mse_scores),
                    numpy.mean(cv_mape_scores),
                    numpy.std(cv_mape_scores)]

    #save and write results + model
    if binary:
        numpy.savetxt(output_dir + 'results\\' + model_name + '-' + str(numpy.mean(cv_accuracy_scores).round(2)) + '.csv', numpy.atleast_2d(measures),
                      delimiter=',', fmt='%6f', header="acc, acc_std, f1, f1_std, precision, precision_std, recall, recall_std, auc, auc_std, auk, auk_std") #write the model scores to a csv file

        if model_type == 'transformer':
            best_model.save_weights(output_dir + 'models\\' + model_name + '_model-weights.h5', save_format='h5') #transformer models can only save weights, not complete models
        else:
            best_model.save(output_dir + 'models\\' + model_name + '.h5')

        text_file = open(output_dir + 'results\\hyperparameters\\' + model_name + "-" + str(numpy.mean(cv_accuracy_scores).round(2)) + ".txt", "w") #write hyperparameters of best run
        text_file.write(str(best_run))
        text_file.close()
    else:
        numpy.savetxt(output_dir + 'results\\' + model_name + '-' + str(numpy.mean(cv_mae_scores).round(2)) + '.csv', numpy.atleast_2d(measures),
                      delimiter=',', fmt='%6f', header='mae, mae_std, mse, mse_std, mape, mape_std') #write the model scores to a csv file

        if model_type == 'transformer':
            best_model.save_weights(output_dir + 'models\\' + model_name + '_model-weights.h5', save_format='h5') #transformer models can only save weights, not complete models
        else:
            best_model.save(output_dir + 'models\\' + model_name + '.h5')

        text_file = open(output_dir + 'results\\hyperparameters\\' + model_name + '-' + str(numpy.mean(cv_mae_scores).round(2)) + '.txt', 'w') #write hyperparameters of best run
        text_file.write(str(best_run))
        text_file.close() 



## Loop for all combinations
function below combines all functions into a single function

In [10]:
def pasqua_et_al(file_location, output_dir):
    model_name = file_location.split("\\")[-1:][0].split('.')[0] #get filename (without.csv)
    print('Now starting with dataset: {}'.format(model_name))

    #preprocess and split training/test data, also encode to images with pixel values
    x_train, x_val, x_test, y_train, y_val, y_test, binary, X, y, model_type = prepare_dataset_for_model(file_location, model_type='cnn')
    x_train_images, x_val_images, x_test_images = image_encoder_val(x_train, x_val, x_test, y_train, y_val, y_test)
    
    #optimize the model hyperparameters through hyperas 
    best_run, best_model = optim.minimize(model=create_model,
                                  data=prepare_dataset_for_model,
                                  algo=tpe.suggest,
                                  max_evals=5, #number of "random" parameter configurations that are tested
                                  trials=Trials(),
                                  functions=[image_encoder_val],
                                  data_args=(file_location, 'cnn'), #supply the arguments for the prepare_dataset_for_model function here
                                  eval_space=True,
                                  notebook_name='(Pasquadibisceglie et al., 2020)',
                                  verbose=False)
    
    print("Evalutation of best performing model:")
    best_scores = best_model.evaluate(x_test_images, y_test, verbose=0)
    print(best_scores)
    print(best_model.metrics_names)

    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    #add AUK & Kappa scores and save the best performing optimized model
    if binary:
        y_pred = best_model.predict(x_test_images, verbose=0)
        best_scores.append(AUK(y_test, y_pred.flatten()).calculate_auk())
        best_scores.append(AUK(y_test, y_pred.flatten()).kappa_curve())
        pd.DataFrame(best_scores).transpose().to_csv(output_dir + 'opt_results\\' + model_name + '.csv')
    else:
        pd.DataFrame(best_scores).transpose().to_csv(output_dir + 'opt_results\\' + model_name + '.csv')

    
    #cross validate to obtain reliable performance of best performing model
    cross_validate_best_model(X=X, y=y, best_model=best_model, best_run=best_run, binary=binary, output_dir=output_dir, model_name=model_name, model_type=model_type)


Finally, generate the results using the code below

In [12]:
output_dir = 'C:\\Users\\20190337\\Downloads\\Tracebook_v2 (Projectfolder)\\model_results\\orange\\'
file_locations = find_all_csv_locations('orange')

for file_location in file_locations:
    pasqua_et_al(file_location, output_dir)

0 csv files left
